In [26]:
import json
all_course_list_url = "https://stevens.smartcatalogiq.com/Institutions/Stevens-Institution-of-Technology/json/2023-2024/Academic-Catalog.json"


In [114]:
import time
import re
from selenium import webdriver
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)  # Optional argument, if not specified will search path.

def run_window(driver, url):
    # elem = WebDriverWait(driver, 30).until(
    #     EC.presence_of_element_located((By.ID, "main")) #This is a dummy element
    # )
    driver.get(url)
    time.sleep(4) # Let the user actually see something!
    search_box = driver.find_element(By.ID, 'main')
    try:
        course_title = search_box.find_element(By.TAG_NAME, 'h1').text
    except Exception as e:
        course_title = ""
    try:
        course_description = search_box.find_element(By.CLASS_NAME, 'desc').text
    except Exception as e:
        course_description = ""
    try:
        course_credits = search_box.find_element(By.CLASS_NAME, 'credits').text
        
    except Exception as e:
        course_credits = ""
    try:
        course_prereqs = search_box.find_element(By.CLASS_NAME, 'sc_prereqs').text
        regex = r"[A-Z]{2,4}\s\d{3}"
        course_prereqs = [match.group(0) for match in re.finditer(regex, course_prereqs)]
        
    except Exception as e:
        course_prereqs = ""
    try:
        course_distribution = search_box.find_element(By.ID, 'distribution').text
    except Exception as e:
        course_distribution = ""
    try:
        
        course_offering = search_box.find_element(By.TAG_NAME, 'p').text
    except Exception as e:
        course_offering = ""


    # time.sleep(1) # Let the user actually see something!
    offers = []
    if "Fall" in course_offering:
        offers.append("Fall")
    if "Spring" in course_offering:
        offers.append("Spring")
    if "Summer Session 1" in course_offering:
        offers.append("Summer Session 1")
    if "Summer Session 2" in course_offering:
        offers.append("Summer Session 2")
        
    return {
        'course_title': course_title,
        'course_description': course_description,
        'course_credits': course_credits,
        'course_prereqs': course_prereqs,
        'course_offered_in': offers
    }

In [115]:
data_ = json.load(open("courses.json", "r"))

In [121]:

base_url = "https://stevens.smartcatalogiq.com/en"
courses_list_idx = 24 

all_courses = data_['Children'][courses_list_idx]
all_courses = all_courses['Children']

file = open("all_cs.json", "a")
file.write("[")
for idx, c in enumerate(all_courses):
  print(f"{idx}. You're looking at department: {c['Name']}")

  for idx, department in enumerate(c['Children']):
    print(f"{idx}. You're looking at course difficulty: {department['Name']}")

    for idx, course in enumerate(department['Children']):
      
      course_detail_url = f"{base_url}{course['Path']}"
      
      print(f"Fetching data for {course['Name']}")
      data = run_window(driver, course_detail_url)
      data['course_code'] = course['Name']
      data['course_level'] = c['Name']
      data['course_rating'] = None
      data['course_professors'] = []
      data['course_syllabus'] = ""
      data['currently_enrolled'] = []
      data['previous_enrolled'] = []
      data['stevens_course_link'] = f"{base_url}{course['Path']}"
      file.write(json.dumps(data) + ',')
file.write("]")
file.close()

0. You're looking at department: AAI - Applied Artificial Intelligence
0. You're looking at course difficulty: 500
Fetching data for AAI 551
1. You're looking at course difficulty: 600
Fetching data for AAI 627
Fetching data for AAI 628


KeyboardInterrupt: 